
    Необходимо загрузить исходный набор данных и соответствующие метки классов.
    Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.
    Написать модель нейронной сети для решения задачи классификации.
    Описать пайплайн предобработки данных. ВАЖНО: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).
    Написать train loop (цикл обучения). Провести эксперименты по обучению с различными значениями параметров learning rate (скорость обучения) и batch size (размер мини-пакета). Выбрать по 3 значения для learning rate и batch size (итоговое количество экспериментов будет 9).
    Для каждого проведенного эксперимента вывести графики для значения функции потерь (ось x - итерация обучения/номер эпохи; ось y - значение функции потерь) и выбранной метрики качества (ось x - итерация обучения/номер эпохи; ось y - значение метрики качества). Графики необходимо выводить как для обучающей, так и для валидационной выборки.
    Оценить качество работы модели на тестовой выборке.
    Сделайте выводы по полученным результатам проведенных экспериментов. Какую модель из всех полученных стоит использовать?
    Сохранить обученную модель.
    Выполните повторную инициализацию модели и загрузку весов. Продемонстрируйте работоспособность модели (пропустите через нее какой-то отзыв/рецензию и выведите результат).



In [2]:
import pandas as pd
from numpy import loadtxt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from tqdm import trange

from pymystem3 import Mystem

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import random


In [10]:
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Необходимо загрузить исходный набор данных и соответствующие метки классов.
from google.colab import files
files.upload()


In [6]:
# Описать пайплайн предобработки данных. ВАЖНО: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).
data = pd.read_csv("dataframe.csv")
data.dropna(inplace=True)
remove_non_alphabets = lambda x: re.sub(r'[^а-яА-Я]',' ',str(x))
tokenize = lambda x: word_tokenize(x, language = "russian")


def Lemmatize(src: list):
    '''Lemmatizes the passed dataset'''
    text_nomalized = ' '.join(src).lower() 
    m = Mystem()
    lemmas = m.lemmatize(text_nomalized)
    return lemmas


print("remove_non_alphabets")
data['review'] = data["review"].apply(remove_non_alphabets)
print("tokenize")
data["review"] = data["review"].apply(tokenize)
print("lemmatize")
for i in data.index:
    data["review"][i] = Lemmatize(data["review"][i])
print("complete")
data["review"] = data["review"].apply(lambda x: ' '.join(x))
data.dropna(inplace=True)
print(data)


remove_non_alphabets
tokenize
lemmatize


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz
<ipython-input-6-5566d0aae6a2>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["review"][i] = Lemmatize(data['review'][i])


complete
      mark                                             review
0      1.0  однажды   вечер   вечер   вечеромпересматриват...
1      1.0  на   фон   уносить   ветер   который   я   чит...
2      1.0  ну   по   сравнение   с   этот   книга   тот  ...
3      1.0  уже   очень   давно   следить   за   потуга   ...
4      1.0  тропа   у   он   одинаковый   летний   ребенок...
...    ...                                                ...
4995   5.0  какой   удивительный   работа   проделывать   ...
4996   5.0  в   очередной   раз   мочь   написать   сво   ...
4997   5.0  смешной   история   о   то   как   два   жена ...
4998   5.0  небольшой   занимательный   рассказ   этакий  ...
4999   5.0  как   жаль   что   с   скляренко   не   успева...

[4998 rows x 2 columns]


In [8]:
# Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.
frac_seed = random.randint(0,10)
train_df = data.sample(frac=0.9, random_state=frac_seed, ignore_index=True)
valid_df = data.sample(frac=0.1, random_state=frac_seed, ignore_index=True)
for item in valid_df["review"]:
    while item in train_df['review']: 
        tmp = data.sample()
        item["review"] = tmp['review']
        item["mark"] = tmp["mark"]
    
print(data.shape)
print(train_df.shape)
print(valid_df.shape)


(4998, 2)
(4498, 2)
(500, 2)


In [13]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 6)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x


x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
